## 1.1

Q: Determine if a string has all unique characters.

In [1]:
def all_unique(s):  return len(set(s)) == len(s)

In [2]:
import unittest

class TestUnique(unittest.TestCase):
    def testZeroLength(self):  self.assertEqual(all_unique(""), True)
    def testUnique(self):  self.assertEqual(all_unique("a"), True)
    def testNoneUnique(self):  self.assertEqual(all_unique("b"), True)

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


## 3.1

Q: Describe how to implement three stacks using a single array.

A: The solution is to maintain three integer values, one for the highest element of each stack, with the integers starting at $\{1, 2, 3\}$ and incrementing by one. Elements go in the array as:

$$\{ s_{1,1}, s_{2, 1}, s_{3, 1}, s_{1, 2}, s_{2, 2}, s_{3, 2}, \ldots \}$$

## 3.2

Q: How would you design a stack that also has an $O(1)$ `min` operation?

A: By maintaing an invariant pointer $p$ which always points to the minimum element in the stack. Comparisons for potential updates on new value inserts are linear time, so this additional operation can be maintained with additional $O(1)$ space and a non-significant tax on other operation times.

We can maintain a list of as many values as necessary, up to the length of the entire list, depending on the target usage pattern. E.g. we can maintain the three lowest elements, so that if the two lowest get popped we still have the last lowest stored. If we run out of minimum values in the last we will have to refill the list, which is $O(n)$. If we choose a good number for our usage pattern however we can still expect $O(1)$ amortized time.

## 3.3

Q: Implement a `SetOfStacks` which implements a stack split across several substacks. Then, implement a `popAt` operation that removes the topmost element from a specific substack.

In [2]:
import math


"""Not an efficient stack implementation but that's not the point of the exercise."""
class Stack:
    def __init__(self, values=None):
        self.store = list(values) if values else []
        self.length = len(values) if values else 0
        
    def pop(self):
        self.length -= 1
        return self.store.pop()
    
    def push(self, v):
        self.length += 1
        self.store.append(v)


class SetOfStacks:
    def __init__(self, values=None, stack_size=3):
        self.store = []
        self.length = 0
        self.stack_size = stack_size
        for v in values:
            self.push(v)
            
    def push(self, v):
        if self.length % self.stack_size == 0:
            next_stack = Stack([v])
            self.store.append(next_stack)
        else:
            self.store[-1].push(v)
        self.length += 1
    
    def pop(self):
        v = self.store[math.ceil(self.length / self.stack_size) - 1].pop()
        self.length -= 1
        
        if self.length % self.stack_size == 0:
            self.store = self.store[:-1]
        
        if v == None:
            return self.pop()
        else:
            return v
    
    def popAt(self, iloc):
        v = self.store[iloc].pop()
        self.store[iloc].push(None)
        return v

In [33]:
import unittest

class TestSetOfStacks(unittest.TestCase):
    def testStackPop(self):
        s = Stack([1, 2, 3])
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 2)
        self.assertEqual(s.pop(), 1)
        
    def testStackPush(self):
        s = Stack()
        s.push(1)
        s.push(2)
        s.push(3)
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 2)
        self.assertEqual(s.pop(), 1)
        
    def testSetOfStacksPop1(self):
        s = SetOfStacks(values=[1, 2, 3], stack_size=1)
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 2)
        self.assertEqual(s.pop(), 1)
        
    def testSetOfStacksPop3(self):
        s = SetOfStacks(values=[1, 2, 3], stack_size=3)
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 2)
        self.assertEqual(s.pop(), 1)

    def testSetOfStacksPop10(self):
        s = SetOfStacks(values=[1, 2, 3], stack_size=10)
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 2)
        self.assertEqual(s.pop(), 1)
    
    def testSetOfStacksPop10(self):
        s = SetOfStacks(values=[1, 2, 3], stack_size=10)
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 2)
        self.assertEqual(s.pop(), 1)    
    
    def testSetOfStacksPush(self):
        s = SetOfStacks(values=[], stack_size=1)
        s.push(1)
        s.push(2)
        s.push(3)
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 2)
        self.assertEqual(s.pop(), 1)    
    
    def testSetOfStacksPopAt(self):
        s = SetOfStacks(values=[1, 2, 3], stack_size=1)
        self.assertEqual(s.popAt(0), 1)
        self.assertEqual(s.popAt(1), 2)
        self.assertEqual(s.popAt(2), 3)
        
    def testSetOfStacksPopAtPopInteraction1(self):
        s = SetOfStacks(values=[1, 2, 3], stack_size=1)
        self.assertEqual(s.popAt(2), 3)
        self.assertEqual(s.pop(), 2)

    def testSetOfStacksPopAtPopInteraction2(self):
        s = SetOfStacks(values=[1, 2, 3], stack_size=1)
        self.assertEqual(s.popAt(1), 2)
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.pop(), 1)
        
if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

.........
----------------------------------------------------------------------
Ran 9 tests in 0.011s

OK


This implementation uses a list, which doesn't slow down the amortized time of the algorithm but does slow it down in practice. An implementation using linked lists for storing the stacks and for storing the stack values would be faster in theory.

With this coda, maintaing separate substacks doesn't modify the properties of the stack meaningfully: push and pop are still both $O(1)$.

The addition of `popAt` introduces the possibility of empty slots in the substacks. Maintaining empty slots is faster than copying substack values. When we go to refund the value, we can inspect if the value is a `None`, and track back recursively until we get a non-`None` value. This is still $O(1)$ amortized time.

## 3.4

Q: Implement a queue using two stacks.

This is a data structures trivia question and not something you would ever do in production! Sheesh.

You can do this by placing the latest element on the left stack, then placing the contents of the right stack on top of the left stack, then rotating the left and right stacks (swapping their positions).

The enqueue algorithm is $O(1)$ if we maintain a pointer to the root of the right substack, and $O(n)$ if we do not (we must iterate over the stack elements to find its bottom to move it to the left substack).

The dequeue algorithm is still $O(1)$.

In [34]:
# Again we are using lists, not as loog as linked lists but not the point.

class TwoStackQueue(unittest.TestCase):
    def __init__(self):
        self.left = Stack()
        self.right = Stack()
        
    def enqueue(v):
        self.left = Stack(self.right.store + [v])
        self.left, self.right = self.right, self.left
        
    def dequeue():
        return self.right.pop()
        
# Note: skipping the tests on this one.

## 4.1

Q: Given a direct graph, design an algorithm to determine whether or not there is a route between two nodes.

In [1]:
class Node:
    def __init__(self, uvalue):
        self.links = []
        self.value = uvalue
    
    def link(self, n):
        self.links.append(n)
        

def bfs_with_stop_condition(source, target):
    next_nodes = [source]
    ignore_list = set()
    
    while True:
        if len(next_nodes) == 0:
            return False
        
        next_next_nodes = []
        for next_node in next_nodes:
            if next_node.value == target.value:
                return True
            
            next_next_nodes += [node for node in next_node.links if node.value not in ignore_list]
        
        ignore_list.update([n.value for n in next_nodes])
        next_nodes = next_next_nodes

In [66]:
A = Node(1)
B = Node(2)
C = Node(3)
A.link(B)

bfs_with_stop_condition(A, B)

True

This is BFS with a stop condition. The algorithm has an average-case performance of $O(k^c)$, where $k$ is the average node interconnectivity and $c$ is the half the distance from the root node to the furthest edge of the graph.

A better algorithm would be a bipartite search, which would emminate from both the source node and the target node and maintain a list of nodes that both sides has seen. We iterate the source search first, then the target search, then the source search, and so on. We have a solution when we find a node in one search that has already appeared in the other search. This will roughly halve the average-case running time to $O(k^{c/2})$.

## 4.2

Q: *Binary search tree allocation* &mdash; Create a binary search tree of minimum height for a sorted array of unique integers.

In [86]:
def binarize(values):
    if len(values) == 0:
        return None
    elif len(values) == 1:
        return Node(values[0])
        
    l = len(values)
    root = Node(values[l // 2])
    root.links.append(binarize(values[:l // 2]))
    root.links.append(binarize(values[l // 2:]))
    return root

Q: Given a binary tree, create a linked list of nodes at each depth.

The solution is to perform any search you want, incorporating tracking on what depth you are looking at a node.

In [101]:
def bfs_with_depth(root):
    next_nodes = [root]
    out = []
    current_depth = 0
    
    while True:
        if len(next_nodes) == 0:
            return out
        
        next_next_nodes = []
        for next_node in next_nodes:
            next_next_nodes += next_node.links
        
        out.append(next_nodes)
        current_depth += 1
        next_nodes = next_next_nodes

In [107]:
import unittest

class TestDepthOrdering(unittest.TestCase):
    def test(self):
        A = Node(1)
        B = Node(2)
        C = Node(3)
        D = Node(4)
        A.link(B)
        A.link(C)
        C.link(D)
        self.assertEqual(bfs_with_depth(A), [[A], [B, C], [D]])
        
if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.005s

OK


This algorithm is $O(n)$, where n is the number of nodes. This is the best case algorithm as there's no way to avoid visiting every single node (except that linked lists would be faster than lists here).

## 3.5

Q: *Sort stack* &mdash; Implement a stack that retains sort order. You may use an additional stack but cannot use any other data structure.

In [32]:
class SortStack:
    def __init__(self):
        self.stack = Stack()
        self.aux = Stack()
        self.length = 0
        
    def push(self, v):
        if self.length == 0:
            self.stack.push(v)
            self.length = 1
        else:
            while self.stack.length > 0:
                prior = self.stack.pop()
                if prior < v:
                    self.stack.push(prior)
                    break
                else:
                    self.aux.push(prior)
            
            self.stack.push(v)
            
            while self.aux.length > 0:
                self.stack.push(self.aux.pop())

In [33]:
s = SortStack()
s.push(5)
s.push(3)
s.push(9)

In [34]:
s.stack.store

[3, 5, 9]

Comment: a relatively easy solution. This push algorithm is $O(1)$ in the best case (exactly ordered inserts take linear time) and $O(n)$ in the worst case (exactly backwards ordered inserts always take the full length of the existing stack).

Because we can only use auxillary stack elements there isn't a simple solution that's better.

Addendum: the question actually asks to *convert an existing stack*. The basic idea is the same, but I rather misinterpreted this one...

## 4.3

Q: Given a binary tree, design an algorithm that creates a linked list of values at each depth.

For simplification I'll just use a list here.

One way to do this is to perform BFS with an additional save step.

In [44]:
def linkify(root):
    next_nodes = [root]
    nodes_at_depths = []
    next_next_nodes = []        
    
    while len(next_nodes) > 0:
        for node in next_nodes:
            next_next_nodes += [link for link in node.links if link != None]
        nodes_at_depths += [next_nodes]
        next_nodes = next_next_nodes
        next_next_nodes = []
        
    return nodes_at_depths

In [45]:
A = Node(1)
B = Node(2)
C = Node(3)
A.link(B)
A.link(C)

linkify(A)

[[<__main__.Node at 0x7fe254044a58>],
 [<__main__.Node at 0x7fe254044828>, <__main__.Node at 0x7fe2540449b0>]]

Comment: I definitely did this problem before but lost it when I banged up that merge. =/

## 4.4

Omitted, I did it earlier pretty sure.

## 4.5

Q: Implement a check on whether a binary tree is a binary search tree.

Every node in a binary tree maintains a contract with every prior node in its path. The contract is that the given node appears on the same branch of that prior node's tree as it would if it were a direct child. So to verify BST-ness we need to verify that this invariant is satisfied.

In [38]:
def is_bst(root, priors=[]):
    if not root:
        return True
    
    left  = root.links[0] if len(root.links) >= 1 else None
    right = root.links[1] if len(root.links) >= 2 else None
    
    if any([root.value >= value if position == "right" else root.value <= value for (value, position) in priors]):
        return False
    
    else:
        return (True and
            is_bst(left,  priors + [(root.value, "right")]) and
            is_bst(right, priors + [(root.value, "left")]))

In [40]:
import unittest

class TestUnique(unittest.TestCase):
    def testEmpty(self):  self.assertEqual(is_bst(None), True)
        
    def testIsBST3(self):
        A = Node(2)
        B = Node(1)
        C = Node(3)
        A.link(B)
        A.link(C)
        self.assertEqual(is_bst(A), True)
        
    def testIsNotBST3(self):
        A = Node(1)
        B = Node(2)
        C = Node(3)
        A.link(B)
        A.link(C)
        self.assertEqual(is_bst(A), False)
        
    def testIsBSTLeftOnly(self):
        A = Node(2)
        B = Node(1)
        A.link(B)
        self.assertEqual(is_bst(A), True)
    
    # Technically given the way we defined a tree this isn't testable without a left child also.
    #     def testIsBSTRightOnly(self):
    #         A = Node(2)
    #         B = Node(3)
    #         A.link(B)
    #         self.assertEqual(is_bst(A), True)
        
    def testIsBSTNodeONly(self):
        A = Node(1)
        self.assertEqual(is_bst(A), True)

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK


Comment: solved this easily this time around.

## 4.6

Q: Given a node in a binary search tree, implement a way of finding the next successive node in that tree by value. You may assume that each node has a link to its parent.

After doodling some I found that within the structure of a binary search tree, to answer this ask, with respect to the current node, we need (1) the leftmost element of the right subtree, or, if no subtree exists, (2) the first right parent.

In [47]:
class BinaryNode:
    def __init__(self, value, left=None, right=None, parent=None):
        self.value = value
        self.left = left
        self.right = right
        self.parent = parent

In [35]:
def findSuccessive(root):
    if root.right:
        child = root.right
        while child.left:
            child = child.left
        return child
    else:
        parent = root.parent
        while parent.value < root.value:
            parent = parent.parent
        return parent

In [31]:
A = BinaryNode(2)
B = BinaryNode(1, parent=A)
C = BinaryNode(3, parent=A)
A.left = B
A.right = C

findSuccessive(A)

In [38]:
import unittest

class TestFindSuccessive(unittest.TestCase):
    def setUp(self):
        A = BinaryNode(2)
        B = BinaryNode(1, parent=A)
        C = BinaryNode(3, parent=A)
        A.left = B
        A.right = C
        self.root = A
    
    def testRightDown(self):  
        self.assertEqual(findSuccessive(self.root).value, 3)
        
    def testLeftUp(self):  
        self.assertEqual(findSuccessive(self.root.left).value, 2)
        

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


Comment: easy enough.

## 4.7

Q: Find a build order for a given set of projects and dependencies.

This question basically asks to build a basic directed graph task scheduler, a la Airflow and friends. It boils down to solving for the topological sort of a directed graph, if one exists.

In [42]:
def solveDependencies(projects, dependencies):
    projects_map = {project: Node(project) for project in projects}
    
    for (requires, requirement) in dependencies:
        projects_map[requires].link(projects_map[requirement])
    
    projects_with_dependencies = set()
    for project in projects_map.values():
        projects_with_dependencies.update(set([project for project in project.links]))
    projects_without_dependencies = set(projects_map.values()).difference(projects_with_dependencies)
    
    root = list(projects_without_dependencies)[0]

    return bfs(root)

In [44]:
solveDependencies(['A', 'B', 'C'], [['A', 'B'], ['B', 'C']])

This algorithm will not work if there are multiple root nodes without dependencies, but an easy modification can be made to account for this case.

The algorithm iterates over the $k$ dependencies and the $n$ nodes, then does BFS to get a topological sort (spanning tree of this directed graph), again touching $n$ nodes this way. Thus this algorithm is $O(n)$ overall.

Comment: I belive this code would be simpler if I linked up the projects map the other way. I should have penciled this one out on paper first, before jumping into this code, in order to avoid doing this backwards.

## 4.8

Q: Design an algorithm to find the first common ancestor of two nodes in a binary tree (not necessarily a binary search tree).

I assume for this exercise that each node retains a pointer to its parent. In that case solving for this is doing breadth first search, but from two starting points.

In [59]:
class BinaryNode:
    def __init__(self, value, left=None, right=None, parent=None):
        self.value = value
        self.left = left
        self.right = right
        self.parent = parent
        
        
def first_common_ancestor(A, B):
    A_parents = set()
    B_parents = set()
    
    while True:
        A_up = A.parent if A.parent else None
        B_up = B.parent if B.parent else None
        
        if A_up in B_parents:
            return A_up
        elif B_up in A_parents:
            return B_up
        elif not A_up and not B_up:
            return None        
        else:
            if A_up:
                A_parents.update({A_up})
            if B_up:
                B_parents.update({B_up})

In [67]:
import unittest

class TestFindSuccessive(unittest.TestCase):
    def testSimpleNoMatch(self):
        B = BinaryNode('B')
        C = BinaryNode('C')
        self.assertEqual(first_common_ancestor(B, C), None)
        
    def testSimpleTree(self):  
        A = BinaryNode('A')
        B = BinaryNode('B')
        C = BinaryNode('C')
        A.left = B
        B.parent = A
        A.right = C
        C.parent = A
        self.assertEqual(first_common_ancestor(B, C).value, 'A')

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


Comment: done well enough. $O(k)$ overall, where $k$ is the average depth of the tree.

## 4.9

Q: A binary search tree was created by iterating through a list and inserting each element. Return all possible arrays that could result in the construction of a particular BST.

In [132]:
def weave(left, right):
    if not left:
        return [right]
    elif not right:
        return [left]
    else:
        left_subresult = [[left[0]] + w for w in weave(left[1:], right)]
        right_subresult = [[right[0]] + w for w in weave(left, right[1:])]
        return left_subresult + right_subresult

    
def enumerateSources(root, priors=None):
    if not priors:
        priors = []
    
    if not root.left and not root.right:
        return [[root.value] + prior for prior in priors] if priors else [root.value]
    else:
        left  = enumerateSources(root.left,  priors=priors)
        right = enumerateSources(root.right, priors=priors)
        return [[root.value] + prior for prior in weave(left, right)]

In [134]:
weave([1, 2], [3, 4, 5])

[[1, 2, 3, 4, 5],
 [1, 3, 2, 4, 5],
 [1, 3, 4, 2, 5],
 [1, 3, 4, 5, 2],
 [3, 1, 2, 4, 5],
 [3, 1, 4, 2, 5],
 [3, 1, 4, 5, 2],
 [3, 4, 1, 2, 5],
 [3, 4, 1, 5, 2],
 [3, 4, 5, 1, 2]]

In [133]:
A = BinaryNode('A')
B = BinaryNode('B')
C = BinaryNode('C')
A.left = B
B.parent = A
A.right = C
C.parent = A

enumerateSources(A)

[['A', 'B', 'C'], ['A', 'C', 'B']]

This is an extremely hard problem. I was able to solve for the shape of the solution very quickly. I was able to identify the weave subproblem very quickly. I got the general form of `enumerateSources` correct immediately. I got the general form of the `weave` correct, but stumbled around in the list concatenation structure. Overall, for a problem this hard, was excellent...but I know I cannot reliably push through problems this difficult!

## 4.10

Q: Determine if `T2` is a subtree of `T1`.

Here's a working brute-force solution.

In [150]:
def nodes_in_bfs_order(root):
    if not root.left and not root.right:
        return [root]
    else:
        left_sublist  = nodes_in_bfs_order(root.left) if root.left  else []
        right_sublist = nodes_in_bfs_order(root.right) if root.right else []
        return [root] + left_sublist + right_sublist
    
def is_match(A, B):
    return nodes_in_bfs_order(A) == nodes_in_bfs_order(B)
    
def is_subtree(T1, T2):
    for T1_sub in nodes_in_bfs_order(T1):
        if is_match(T1_sub, T2):
            return True
    return False

In [155]:
A = BinaryNode('A')
B = BinaryNode('B')
C = BinaryNode('C')
A.left = B
B.parent = A
A.right = C
C.parent = A
D = BinaryNode('D')

In [159]:
import unittest

class TestIsSubtree(unittest.TestCase):
    def test(self):
        A = BinaryNode('A')
        B = BinaryNode('B')
        C = BinaryNode('C')
        A.left = B
        B.parent = A
        A.right = C
        C.parent = A
        
        D = BinaryNode('D')
        
        self.assertEqual(is_subtree(A, A), True)
        self.assertEqual(is_subtree(A, B), True)
        self.assertEqual(is_subtree(A, C), True)
        self.assertEqual(is_subtree(A, D), False)

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


This algorithm is $O(n^2)$.

It could be improved by making smart optimizations around the path of the BFS, which could drop us to $O(n)$.

The alternative approach, which I considered but didn't followed, would be to transform this problem into a string comparison problem by hashing the trees. If you can hash the trees in such a way that you preserve order, you can do a string comparison afterwards, which would reduce the problem to $O(n)$. You can do this if you include `null` values! Totally could have come up with this one.

Anyway, this implementation is fine (albeit slower) and I did it pretty efficiently. Next!

## 4.11

This question asks you to implement a binary tree that allows random node retrieval. Meh, decided to skip it.

## 4.12

Q: Finds all paths of any length within a binary tree with integer values which have a certain sum.

This is BFS or DFS whilst keeping track of previous `{sum, directions}` tuples.

In [178]:
def append_to_priors(new, priors):
    # TODO: this can be done without copying the dict, but I'm in a rush.
    next_priors = priors.copy() if priors else dict()
    
    for prior_sum in next_priors.keys():
        for path in next_priors[prior_sum]:
            next_value = new.value + prior_sum
            next_path = path + [new]
            
            if next_value in next_priors.keys():
                next_priors[next_value] += next_path
            else:
                next_priors[next_value] = [next_path]
    return next_priors


def integer_sum_paths(root, sumval, priors=None):
    if not priors:
        prior_paths_by_sum = dict()
    
    if sumval - root.value in prior_paths_by_sum.keys():
        out = [root]
        # TODO
    else:
        out = []
    
    if root.left:
        out = out + integer_sum_paths(root.left, sumval, priors=append_to_priors(root, priors))
    elif root.right:
        out = out + integer_sum_paths(root.right, sumval, priors=append_to_priors(root, priors))
        
    return out

In [174]:
A = BinaryNode(1)
B = BinaryNode(2)
C = BinaryNode(3)
A.left = B
B.parent = A
A.right = C
C.parent = A

In [171]:
append_to_priors(BinaryNode(2), {1: [[BinaryNode(1)]]})

{1: [[<__main__.BinaryNode at 0x7feff32e0f98>]],
 3: [[<__main__.BinaryNode at 0x7feff32e0f98>,
   <__main__.BinaryNode at 0x7feff32e0668>]]}

A corrected version of this algorithm is the solution (I ran out of time!). The algorithm is $O(n)$.

In [179]:
integer_sum_paths(A, 3)

[]

This algorithm is $O(nk)$, where $n$ is the number of nodes and $k$ the average depth, because we must iterate over each node and, on each iteration, copy some data from nodes previously on the path.